In [1]:
from __future__ import division, print_function, absolute_import
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50


In [2]:
cd F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn

F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn


In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
# 图片路径，两组标签都在该目录下
cwd = r".\train224\resize224"

# tfrecord文件保存路径
file_path = r"./"

# 每个tfrecord存放图片个数
bestnum = 1000

# 第几个图片
num = 0

# 第几个TFRecord文件
recordfilenum = 0

# 将labels放入到classes中
classes = [str(i) for i in range(37)]
print(classes)

# tfrecords格式文件名
ftrecordfilename = ("traindata_cat&dog_no_augmented.tfrecords-%.3d" % recordfilenum)
writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']


In [7]:
for index, name in enumerate(classes):

    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num = num + 1
        if num > bestnum:  # 超过100，写入下一个tfrecord
            num = 1
            recordfilenum += 1
            ftrecordfilename = ("traindata_cat&dog_no_augmented.tfrecords-%.3d" % recordfilenum)
            writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))
        img_path = os.path.join(class_path, img_name)  # 每一个图片的地址
        img = Image.open(img_path, 'r')
        img_raw = img.tobytes()  # 将图片转化为二进制格式
        example = tf.train.Example(
            features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            }))
        writer.write(example.SerializeToString())  # 序列化为字符串
writer.close()

In [4]:
def read_and_decode_tfrecord(filename):
    filename_deque = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_deque)
    features = tf.parse_single_example(serialized_example, features={
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string)})
    label = tf.cast(features['label'], tf.int32)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])
    img = tf.cast(img, tf.float32) / 255.0      #将矩阵归一化0-1之间
    return img, label

In [9]:
os.listdir()

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'cats only',
 'cats tfrecord',
 'dogs only',
 'dogs tfrecord',
 'resize224',
 'resize224.zip',
 'resize224_augmented.zip',
 'resnet50.hdf5',
 'test224',
 'test224.zip',
 'train224',
 'train224.zip',
 'traindata_cat&dog_no_augmented.tfrecords-000',
 'traindata_cat&dog_no_augmented.tfrecords-001',
 'traindata_cat&dog_no_augmented.tfrecords-002',
 'traindata_cat&dog_no_augmented.tfrecords-003',
 'traindata_cat&dog_no_augmented.tfrecords-004',
 'traindata_cat&dog_no_augmented.tfrecords-005',
 'traindata_cat&dog_no_augmented.tfrecords-006',
 'traindata_cat&dog_no_augmented.tfrecords-007']

In [5]:
train_list = ['traindata_cat&dog_no_augmented.tfrecords-000',
 'traindata_cat&dog_no_augmented.tfrecords-001',
 'traindata_cat&dog_no_augmented.tfrecords-002',
 'traindata_cat&dog_no_augmented.tfrecords-003',
 'traindata_cat&dog_no_augmented.tfrecords-004',
 'traindata_cat&dog_no_augmented.tfrecords-005',
 'traindata_cat&dog_no_augmented.tfrecords-006',
 'traindata_cat&dog_no_augmented.tfrecords-007']

In [6]:
# Training Parameters
learning_rate = 0.0001
num_steps = 1
batch_size = 32
display_step = 10

# Network Parameters
num_input = 50176
num_classes = 37

# tf Graph input
X = tf.placeholder(tf.float32, [None, 224, 224, 3])
Y = tf.placeholder(tf.float32, [None])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [7]:
img, label = read_and_decode_tfrecord(train_list)
img_batch, label_batch = tf.train.shuffle_batch([img, label], num_threads=32, batch_size=batch_size, capacity=10000,min_after_dequeue=9900)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [8]:
# 将label值进行onehot编码
one_hot_labels = tf.one_hot(indices=tf.cast(Y, tf.int32), depth=37)


In [9]:
input = layers.Input((224, 224, 3))
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model(input)
x = layers.Flatten()(x)
prediction = layers.Dense(num_classes,activation='softmax')(x)
resnet50 = Model(input, prediction)
resnet50.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [11]:
# Start training
saver = tf.train.Saver(max_to_keep=1)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
counter=0
# Run the initializer
for epoch in range(1, 6):
    print("epoch:", epoch)
    with tf.device('/gpu:0'):
        for batch in range(1,221): #47208 pics /7078
            batch_x, batch_y = sess.run([img_batch, label_batch])
            batch_y = sess.run(one_hot_labels, feed_dict={Y: batch_y})
            loss, acc = resnet50.train_on_batch(batch_x, batch_y)
            print("Epoch " + str(epoch) + ", Minibatch Loss= " + \
                        "{:.4f}".format(loss) + ", Training Accuracy= " + \
                        "{:.3f}".format(acc))
        resnet50.save_weights('resnet50.hdf5')
print("Optimization Finished!")
coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
epoch: 1
Instructions for updating:
Use tf.cast instead.
Epoch 1, Minibatch Loss= 5.5134, Training Accuracy= 0.000
Epoch 1, Minibatch Loss= 5.7308, Training Accuracy= 0.094
Epoch 1, Minibatch Loss= 5.0606, Training Accuracy= 0.094
Epoch 1, Minibatch Loss= 7.0088, Training Accuracy= 0.062
Epoch 1, Minibatch Loss= 6.4886, Training Accuracy= 0.094
Epoch 1, Minibatch Loss= 4.2292, Training Accuracy= 0.031
Epoch 1, Minibatch Loss= 5.1314, Training Accuracy= 0.156
Epoch 1, Minibatch Loss= 4.7076, Training Accuracy= 0.219
Epoch 1, Minibatch Loss= 3.8352, Training Accuracy= 0.375
Epoch 1, Minibatch Loss= 4.0134, Training Accuracy= 0.188
Epoch 1, Minibatch Loss= 3.1935, Training Accuracy= 0.281
Epoch 1, Minibatch Loss= 4.3730, Training Accuracy= 0.281
Epoch 1, Minibatch Loss= 3.0075, Training Accuracy= 0.281
Epoch 1, Minibatch Loss= 3.5422, Training Accuracy= 0.375
Epoch 1, Minibatch Loss= 3.2233, Training Accura

Epoch 1, Minibatch Loss= 0.7739, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.1052, Training Accuracy= 0.969
Epoch 1, Minibatch Loss= 0.5278, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 0.4914, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.6126, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 1.3253, Training Accuracy= 0.719
Epoch 1, Minibatch Loss= 1.1444, Training Accuracy= 0.719
Epoch 1, Minibatch Loss= 0.4056, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.6385, Training Accuracy= 0.750
Epoch 1, Minibatch Loss= 0.5772, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.3187, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.9857, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.4459, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 0.1429, Training Accuracy= 0.969
Epoch 1, Minibatch Loss= 0.4516, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 1.2448, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.8405, Training Accuracy= 0.844
Epoch 1, Minib

Epoch 2, Minibatch Loss= 0.2934, Training Accuracy= 0.875
Epoch 2, Minibatch Loss= 0.5308, Training Accuracy= 0.875
Epoch 2, Minibatch Loss= 0.1708, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.2321, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.0489, Training Accuracy= 1.000
Epoch 2, Minibatch Loss= 0.3012, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.3627, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.1131, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.2550, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.1126, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.1237, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.1343, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.2063, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.3960, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.5449, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.6993, Training Accuracy= 0.812
Epoch 2, Minibatch Loss= 0.4767, Training Accuracy= 0.906
Epoch 2, Minib

Epoch 2, Minibatch Loss= 0.1588, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.0844, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.0253, Training Accuracy= 1.000
Epoch 2, Minibatch Loss= 0.0304, Training Accuracy= 1.000
Epoch 2, Minibatch Loss= 0.1284, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.3606, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.1922, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.0324, Training Accuracy= 1.000
Epoch 2, Minibatch Loss= 0.1706, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.1174, Training Accuracy= 0.938
Epoch 2, Minibatch Loss= 0.0524, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.1266, Training Accuracy= 0.969
Epoch 2, Minibatch Loss= 0.1283, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.0043, Training Accuracy= 1.000
Epoch 2, Minibatch Loss= 0.4511, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.3752, Training Accuracy= 0.906
Epoch 2, Minibatch Loss= 0.2349, Training Accuracy= 0.875
Epoch 2, Minib

Epoch 3, Minibatch Loss= 0.1558, Training Accuracy= 0.938
Epoch 3, Minibatch Loss= 0.0102, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0503, Training Accuracy= 0.969
Epoch 3, Minibatch Loss= 0.0032, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0226, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0786, Training Accuracy= 0.969
Epoch 3, Minibatch Loss= 0.0342, Training Accuracy= 0.969
Epoch 3, Minibatch Loss= 0.1257, Training Accuracy= 0.969
Epoch 3, Minibatch Loss= 0.0228, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0199, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0048, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.0308, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.1581, Training Accuracy= 0.938
Epoch 3, Minibatch Loss= 0.0032, Training Accuracy= 1.000
Epoch 3, Minibatch Loss= 0.2059, Training Accuracy= 0.938
Epoch 3, Minibatch Loss= 0.0669, Training Accuracy= 0.969
Epoch 3, Minibatch Loss= 0.0006, Training Accuracy= 1.000
Epoch 3, Minib

Epoch 4, Minibatch Loss= 0.0292, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0688, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0018, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0129, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.2239, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0147, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0453, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0268, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0035, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0630, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.1129, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0143, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0031, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0321, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0060, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.2327, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 4, Minib

Epoch 4, Minibatch Loss= 0.0043, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0171, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0379, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0674, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.2523, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0020, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0756, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.2427, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0074, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.0094, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.1217, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0094, Training Accuracy= 1.000
Epoch 4, Minibatch Loss= 0.1830, Training Accuracy= 0.969
Epoch 4, Minibatch Loss= 0.0067, Training Accuracy= 1.000
Epoch 4, Minib

Epoch 5, Minibatch Loss= 0.0481, Training Accuracy= 0.969
Epoch 5, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0013, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0008, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0043, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0740, Training Accuracy= 0.969
Epoch 5, Minibatch Loss= 0.0011, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0016, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0048, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0041, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0023, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0052, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0005, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0031, Training Accuracy= 1.000
Epoch 5, Minibatch Loss= 0.0011, Training Accuracy= 1.000
Epoch 5, Minib

In [12]:
counter=0
right=0
for label in os.listdir (r'.\test224'):


    test_dir= os.path.join(r'.\test224', label)

    for pic in os.listdir(test_dir):
        img_path = os.path.join(test_dir, pic)
        img = Image.open(img_path)
        img = img.resize((224, 224))
        img = tf.reshape(img, [1, 224, 224, 3])
        img1 = tf.reshape(img, [1, 224, 224, 3])
        img = tf.cast(img, tf.float32) / 255.0
        b_image, b_image_raw = sess.run([img, img1])
        haha_sm = resnet50.predict(b_image)
        print(sess.run(tf.argmax(haha_sm, 1)),int(label))
        if sess.run(tf.argmax(haha_sm, 1))== int(label):
            right +=1

        counter +=1
acc=right/counter
print(acc)

[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[20] 0
[0] 0
[33] 0
[33] 0
[27] 0
[2] 1
[2] 1
[1] 1
[14] 1
[15] 1
[1] 1
[28] 1
[19] 1
[1] 1
[33] 10
[30] 10
[33] 10
[10] 10
[10] 10
[33] 10
[31] 10
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[30] 12
[22] 12
[12] 12
[12] 12
[12] 12
[12] 12
[12] 12
[12] 12
[16] 13
[13] 13
[19] 13
[13] 13
[12] 13
[13] 13
[13] 13
[28] 13
[14] 14
[3] 14
[14] 14
[14] 14
[28] 14
[14] 14
[14] 14
[8] 14
[15] 15
[15] 15
[35] 15
[15] 15
[29] 15
[22] 15
[15] 15
[15] 15
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[26] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[19] 19
[19] 19
[19] 19
[19] 19
[19] 19
[19] 19
[19] 19
[30] 19
[35] 2
[34] 2
[14] 2
[35] 2
[1] 2
[2] 2
[10] 2
[2] 2
[20] 20
[20] 20
[20] 20
[20] 20
[20] 20
[20] 20
[20] 20
[20] 20
[21] 21
[21] 21
[21] 21
[21] 21
[3] 21
[10] 21
[10] 21
[21] 21
[22] 22
[22] 22
[22] 22
[22] 22
[22] 22
[22] 22
[22] 22
[22] 22
[23]